<a href="https://colab.research.google.com/github/Prathamesh-Chavan-98/weather-and-news-telling-ai-agent/blob/main/_weather_news_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import os
!pip install python-dotenv
!pip install groq-ai
!pip install python-dotenv
from dotenv import load_dotenv
!pip install groq
from groq import Groq

# Load environment variables from .env file
load_dotenv()

# Retrieve the Groq API key
GROQ_API_KEY = "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"
if not GROQ_API_KEY:
    raise ValueError("Groq API key not found. Set GROQ_API_KEY in .env file.")

# Retrieve the News API key
NEWS_API_KEY = "90665c9a286d44bbb31286f202fc62f8"
weather_api_key = "2b15403f6ac54ea7b3353310242712"



import requests

weather_api_key = "2b15403f6ac54ea7b3353310242712"

def get_weather(api_key, location):
    """
    Fetches weather data from the WeatherAPI.
    """
    base_url = "http://api.weatherapi.com/v1/current.json"
    params = {
        "key": api_key,
        "q": location
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None

if __name__ == "__main__":
    location = input("Enter a city name: ")
    weather_data = get_weather(weather_api_key, location)

    if weather_data:
        print(f"Weather in {location}:")
        print(f"Temperature: {weather_data['current']['temp_c']}°C")
        print(f"Condition: {weather_data['current']['condition']['text']}")
        # Add more weather information as needed...
        print(weather_data)
    else:
        print("Could not retrieve weather data.")



# Initialize the Groq client
client = Groq(api_key=GROQ_API_KEY)




def get_news(api_key, topic):
    """
    Fetches news articles from a news API based on the given topic.
    Replace with your actual API endpoint and parameters.
    This is a placeholder.
    """
    url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        return data.get("articles", [])
    except requests.exceptions.RequestException as e:
        print(f"Error fetching news: {e}")
        return []

def get_groq_response(prompt, llm_name="llama-3.3-70b-versatile"):
    """
    Gets a response from the Groq API.
    """
    messages = [{"role": "user", "content": prompt}]
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=llm_name,
    )
    return chat_completion.choices[0].message.content

def generate_news_summary(news_articles, prompt):
    """
    Generates a news summary using the Groq API based on the provided articles and prompt.
    """
    news_data_str = ""
    for i, article in enumerate(news_articles):
        news_data_str += f"Article {i+1}:\n"
        news_data_str += f"Title: {article.get('title', 'No Title')}\n"
        news_data_str += f"Source: {article.get('source', {}).get('name', 'Unknown Source')}\n"
        news_data_str += f"Description: {article.get('description', 'No Description')}\n"
        news_data_str += f"URL: {article.get('url', 'No URL')}\n\n"

    final_prompt = f"""
    You are an AI News Summarizer. You have been provided with the following news articles:

    {news_data_str}

    Based on these articles, please {prompt}
    """
    response = get_groq_response(final_prompt)
    return response

def run_news_agent(user_query):
    """
    Runs the AI news agent. It decides whether to call the news API
    and then summarizes the results.
    """
    # Step 1: Instruct the LLM to decide if it needs to call the tool
    tool_decision_prompt = f"""
    You are an AI agent that can retrieve and summarize news.
    The user has asked: "{user_query}"

    You have access to a tool called 'get_news' which takes a 'topic' as input
    and returns a list of news articles.

    Based on the user's query, do you need to use the 'get_news' tool?
    If yes, please respond with: 'TOOL_CALL: get_news(topic="<the topic>")'
    If no, or if you can answer directly without external information,
    respond with your direct answer.
    """
    tool_decision = get_groq_response(tool_decision_prompt)
    print(f"LLM's tool decision: {tool_decision}")

    if "TOOL_CALL: get_news(topic=" in tool_decision:
        try:
            start_index = tool_decision.find('topic="') + len('topic="')
            end_index = tool_decision.find('")', start_index)
            topic_to_search = tool_decision[start_index:end_index]
            print(f"Calling get_news tool for topic: {topic_to_search}")
            news_articles = get_news(NEWS_API_KEY, topic_to_search)

            if news_articles:
                summary_prompt = f"summarize the key information and main points."
                summary = generate_news_summary(news_articles, summary_prompt)
                return summary
            else:
                return "No relevant news articles found for that topic."

        except Exception as e:
            return f"An error occurred while calling the news tool: {e}"
    else:
        # If the LLM decides not to call the tool, it might try to answer directly
        return get_groq_response(f"The user asked: '{user_query}'. Please provide a direct answer if possible.")

if __name__ == "__main__":
    user_query = input("Ask for news or information: ")
    response = run_news_agent(user_query)
    print(f"Agent's Response: {response}")

ERROR: Could not find a version that satisfies the requirement groq-ai (from versions: none)
ERROR: No matching distribution found for groq-ai
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 2.9 MB/s eta 0:00:00
Enter a city name: karachi
Weather in karachi:
Temperature: 37.3°C
Condition: Mist
{'location': {'name': 'Karachi', 'region': 'Sindh', 'country': 'Pakistan', 'lat': 24.8667, 'lon': 67.05, 'tz_id': 'Asia/Karachi', 'localtime_epoch': 1746875332, 'localtime': '2025-05-10 16:08'}, 'current': {'last_updated_epoch': 1746874800, 'last_updated': '2025-05-10 16:00', 'temp_c': 37.3, 'temp_f': 99.1, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 13.9, 'wind_kph': 22.3, 'wind_degree': 246, 'wind_dir': 'WSW', 'pressure_mb': 1006.0, 'pressure_in': 29.71, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 24, 'cloud': 0, 'feelslike_c': 39.3, 'feelslike_f': 102.7, 'windchill_c': 31.9, 'windchill_f': 89.5, 

In [ ]:
import requests
import requests
import json
import os
!pip install python-dotenv
!pip install groq-ai
!pip install python-dotenv
from dotenv import load_dotenv
!pip install groq
from groq import Groq
import json
import os
from dotenv import load_dotenv
from groq import Groq

ERROR: Could not find a version that satisfies the requirement groq-ai (from versions: none)
ERROR: No matching distribution found for groq-ai


In [ ]:


# Install necessary libraries (these should be run in your terminal, not within the script)
# !pip install python-dotenv
# !pip install groq
# !pip install requests  # Ensure requests is installed

# Load environment variables (replace with your actual API keys)
load_dotenv()
GROQ_API_KEY = "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"
NEWS_API_KEY = "90665c9a286d44bbb31286f202fc62f8"
WEATHER_API_KEY = "2b15403f6ac54ea7b3353310242712"

# Check for API keys
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY is missing. Please set it in your .env file or as an environment variable.")
if not NEWS_API_KEY:
    raise ValueError("NEWS_API_KEY is missing. Please set it in your .env file or as an environment variable.")
if not WEATHER_API_KEY:
    raise ValueError("WEATHER_API_KEY is missing. Please set it in your .env file or as an environment variable.")


def get_weather(api_key, location):
    """
    Fetches current weather data for a given location using the WeatherAPI.com API.

    Args:
        api_key (str): The WeatherAPI.com API key.
        location (str): The location to fetch weather data for (e.g., "Pimpri-Chinchwad", "London").

    Returns:
        dict: A dictionary containing the weather data, or None if an error occurs.
    """
    base_url = "http://api.weatherapi.com/v1/current.json"
    params = {"key": api_key, "q": location}
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data for {location}: {e}")
        return None


def get_news(api_key, topic):
    """
    Fetches news articles for a given topic using the NewsAPI.org API.

    Args:
        api_key (str): The NewsAPI.org API key.
        topic (str): The topic to fetch news articles for (e.g., "AI", "technology", "local news").

    Returns:
        list: A list of news articles, or an empty list if no articles are found or an error occurs.
    """
    url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data.get("articles", [])  # Returns [] if "articles" is not in the JSON
    except requests.exceptions.RequestException as e:
        print(f"Error fetching news for topic '{topic}': {e}")
        return []



def get_groq_response(prompt, llm_name="llama-3.3-70b-versatile"):
    """
    Sends a prompt to the Groq API and returns the generated response.

    Args:
        prompt (str): The prompt to send to the Groq API.
        llm_name (str, optional): The name of the Groq language model to use.
            Defaults to "llama-3.3-70b-versatile".

    Returns:
        str: The text response from the Groq API.
    """
    client = Groq(api_key=GROQ_API_KEY)
    messages = [{"role": "user", "content": prompt}]
    try:
        chat_completion = client.chat.completions.create(
            messages=messages, model=llm_name
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error getting response from Groq: {e}")
        return "Sorry, I could not process your request due to an error with the language model."



def generate_news_summary(news_articles, prompt="summarize the key information"):
    """
    Generates a summary of the provided news articles using a language model.

    Args:
        news_articles (list): A list of news articles (dictionaries) from the News API.
        prompt (str, optional):  A prompt to guide the summarization.
            Defaults to "summarize the key information".

    Returns:
        str: A summarized version of the news articles, or a message indicating no articles.
    """
    if not news_articles:
        return "No news articles to summarize."

    news_data_str = ""
    for i, article in enumerate(news_articles):
        news_data_str += f"Article {i + 1}:\n"
        news_data_str += f"Title: {article.get('title', 'No Title')}\n"
        news_data_str += f"Source: {article.get('source', {}).get('name', 'Unknown Source')}\n"
        news_data_str += f"Description: {article.get('description', 'No Description')}\n"
        news_data_str += f"URL: {article.get('url', 'No URL')}\n\n"

    final_prompt = f"""
    You are an AI News Summarizer. You have been provided with the following news articles:

    {news_data_str}

    Based on these articles, please {prompt}
    """
    return get_groq_response(final_prompt)



def generate_response(user_query):
    """
    Generates a response to the user's query, deciding whether to call the
    get_news or get_weather functions, or both, using the Groq LLM.

    Args:
        user_query (str): The user's query (e.g., "What's the weather in Mumbai?",
            "Tell me the latest news on technology",
            "What's the weather and news in Delhi?").

    Returns:
        str: The agent's response, combining weather and/or news information,
              or a direct answer from the LLM.
    """
    # Analyze the user prompt to determine the appropriate API
    tool_decision_prompt = f"""
    You are an AI agent that can access weather and news information.
    The user asked: "{user_query}"

    You have access to these tools:
    - 'get_news(topic)': Retrieves news articles on a given topic.
    - 'get_weather(location)': Retrieves the current weather for a given location.

    Decide which tool(s) (if any) are needed to best answer the user's query.
    Respond in the following format:

    - If the query is ONLY about weather in a location:
      TOOL_CALL: get_weather(location="<location>")
    - If the query is ONLY about news on a topic:
      TOOL_CALL: get_news(topic="<topic>")
    - If the query requires BOTH weather AND news for a location:
      TOOL_CALL: get_weather(location="<location>")
      TOOL_CALL: get_news(topic="<location> local news")  (Use location as topic for local news)
    - If the query does not require any tools, provide a direct answer.

    Examples:
    - User: "What's the weather in London?"
      Response: TOOL_CALL: get_weather(location="London")
    - User: "Tell me the latest news on AI"
      Response: TOOL_CALL: get_news(topic="AI")
    - User: "What's the weather and news in Delhi?"
      Response: TOOL_CALL: get_weather(location="Delhi")
      TOOL_CALL: get_news(topic="Delhi local news")
    - User: "Tell me about recent developments."
      Response:  Provide a direct answer.
    """

    tool_decision = get_groq_response(tool_decision_prompt)
    print(f"LLM's tool decision: {tool_decision}")

    weather_result = None
    news_summary = None

    # Process tool calls
    if "TOOL_CALL: get_weather" in tool_decision:
        location = tool_decision.split('get_weather(location="')[1].split('")')[0]
        weather_data = get_weather(WEATHER_API_KEY, location)
        if weather_data:
            weather_result = f"Weather in {location}: Temperature: {weather_data['current']['temp_c']}°C, Condition: {weather_data['current']['condition']['text']}."
        else:
            weather_result = f"Could not retrieve weather information for {location}."

    if "TOOL_CALL: get_news" in tool_decision:
        topic = tool_decision.split('get_news(topic="')[1].split('")')[0]
        news_articles = get_news(NEWS_API_KEY, topic)
        news_summary = generate_news_summary(news_articles)

    # Combine results and generate final response
    if weather_result and news_summary:
        return f"{weather_result} {news_summary}"
    elif weather_result:
        return weather_result
    elif news_summary:
        return news_summary
    else:
        #  If no tools were called, get a direct response from the LLM
        return get_groq_response(f"The user asked: '{user_query}'.  Provide a direct answer.")



if __name__ == "__main__":
    user_query = input("Ask me about the news or weather: ")
    response = generate_response(user_query)
    print(f"Agent's Response: {response}")


Ask me about the news or weather: karachi
LLM's tool decision: TOOL_CALL: get_weather(location="Karachi")
TOOL_CALL: get_news(topic="Karachi local news")
Agent's Response: Weather in Karachi: Temperature: 31.9°C, Condition: Sunny. **Summary of India-Pakistan Conflict**

The current situation between India and Pakistan is highly tense, with both countries accusing each other of launching drone strikes and engaging in cross-border military actions. The conflict is centered around the disputed region of Kashmir, which has been a point of contention between the two nuclear-armed nations for decades.

**Key Incidents:**

1. **Drone Strikes:** India and Pakistan have both accused each other of launching drone strikes, with Pakistan claiming to have shot down 25 Indian drones and India saying it "neutralized" Pakistani attempts to strike military targets.
2. **Cross-Border Attacks:** There have been reports of cross-border attacks, with Pakistan saying eight civilians were killed in Indian mi